# <center>OpenAI agent pattern: parallelization</center>

A starter guide for building an agent with parallel LLM and tool calling using the `openai-agents` library.

In this example, we are building a stock research agent. If we want to research 5 stocks, we can force the agent to run multiple tool calls, instead of sequentially.

### Install Libraries

In [5]:
# Install base libraries for OpenAI
!pip install -q openai openai-agents yfinance

# Install optional libraries for OpenInference/OpenTelemetry tracing
!pip install -q arize-phoenix-otel openinference-instrumentation-openai-agents openinference-instrumentation-openai

### Setup Keys

Add your OpenAI API key to the environment variable `OPENAI_API_KEY`.

Copy your Phoenix `API_KEY` from your settings page at [app.phoenix.arize.com](https://app.phoenix.arize.com).

In [2]:
import os
from getpass import getpass

os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"
if not os.environ.get("PHOENIX_CLIENT_HEADERS"):
    os.environ["PHOENIX_CLIENT_HEADERS"] = "api_key=" + getpass("Enter your Phoenix API key: ")

OPENAI_API_KEY = globals().get("OPENAI_API_KEY") or getpass(
    "🔑 Enter your OpenAI API key: "
)
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

### Setup Tracing

In [3]:
from phoenix.otel import register

tracer_provider = register(
    project_name="openai-agents",
    endpoint="https://app.phoenix.arize.com/v1/traces",
    auto_instrument=True,
)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: openai-agents
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



## Creating your agent

In [ ]:
import yfinance as yf
from pprint import pprint

def get_stock_data(ticker_symbol):
    stock = yf.Ticker(ticker_symbol)
    data = stock.info
    return {
        "symbol": data.get("symbol"),
        "current_price": data.get("currentPrice"),
        "market_cap": data.get("marketCap"),
        "sector": data.get("sector"),
        "industry": data.get("industry"),
        "description": data.get("longBusinessSummary"),
        "trailing_pe": data.get("trailingPE"),
        "forward_pe": data.get("forwardPE"),
        "dividend_yield": data.get("dividendYield"),
        "beta": data.get("beta"),
        "fifty_two_week_high": data.get("fiftyTwoWeekHigh"),
        "fifty_two_week_low": data.get("fiftyTwoWeekLow"),
        "fifty_day_moving_average": data.get("fiftyDayAverage"),
        "two_hundred_day_moving_average": data.get("twoHundredDayAverage"),
        "recommendation_key": data.get("recommendationKey"),
        "revenue_growth": data.get("revenueGrowth"),
        "earnings_growth": data.get("earningsGrowth"),
        "profit_margins": data.get("profitMargins"),
    }

pprint(get_stock_data("AAPL"))

{'beta': 1.275,
 'current_price': 195.2159,
 'description': 'Apple Inc. designs, manufactures, and markets smartphones, '
                'personal computers, tablets, wearables, and accessories '
                'worldwide. The company offers iPhone, a line of smartphones; '
                'Mac, a line of personal computers; iPad, a line of '
                'multi-purpose tablets; and wearables, home, and accessories '
                'comprising AirPods, Apple TV, Apple Watch, Beats products, '
                'and HomePod. It also provides AppleCare support and cloud '
                'services; and operates various platforms, including the App '
                'Store that allow customers to discover and download '
                'applications and digital content, such as books, music, '
                'video, games, and podcasts, as well as advertising services '
                'include third-party licensing arrangements and its own '
                'advertising platforms. I

In [11]:
from textwrap import dedent
from agents import Agent, WebSearchTool, Runner, TResponseInputItem, function_tool
from agents.model_settings import ModelSettings
from pprint import pprint

@function_tool
def get_stock_data_tool(ticker_symbol: str) -> dict:
    """
    Get stock data for a given ticker symbol.
    Args:
        ticker_symbol: The ticker symbol of the stock to get data for.
    Returns:
        A dictionary containing stock data such as price, market cap, and more.
    """
    return get_stock_data(ticker_symbol)


research_agent = Agent(
    name="FinancialSearchAgent",
    instructions=dedent("""You are a research assistant specializing in financial topics. Given a stock ticker, use web search to retrieve up‑to‑date context and produce a short summary of at most 50 words. Focus on key numbers, events, or quotes that will be useful to a financial analyst."""),
    model="gpt-4.1",
    tools=[WebSearchTool(), get_stock_data_tool],
    model_settings=ModelSettings(tool_choice="required", parallel_tool_calls=True),
)

user_input = input("Enter the stock tickers you want to research: ")
input_items: list[TResponseInputItem] = [{"content": user_input, "role": "user"}]

orchestrator = await Runner.run(research_agent, input_items)
orchestrator_output = orchestrator.final_output
pprint(orchestrator_output)

('Apple (AAPL): $195.03, market cap $2.91T, TTM P/E 30.4, yield 0.52%. Margins '
 '24.3%. Revenue growth 5.1%. Recommendation: buy.\n'
 '\n'
 'Alphabet (GOOGL): $150.27, market cap $1.83T, TTM P/E 16.8, yield 0.51%. '
 'Margins 30.9%. Revenue growth 12%. Buy.\n'
 '\n'
 'Microsoft (MSFT): $435.41, market cap $3.24T, TTM P/E 33.6, yield 0.77%. '
 'Margins 35.8%. Revenue growth 13.3%. Strong buy.')
